# Recolección de datos

In [1]:
import requests
import pandas as pd

## Clasificación de países por nivel de ingreso

In [187]:
#Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
url = 'http://api.worldbank.org/v2/incomelevel' 
args = {"format":"json", "prefix":"Getdata"}
dic_Income_Level = requests.get(url, params=args)
datos = dic_Income_Level.json()[1]
data = pd.json_normalize(datos)
data[['value', 'id']]

,value,id
0,High income,HIC
1,Not classified,INX
2,Low income,LIC
3,Lower middle income,LMC
4,Low & middle income,LMY
5,Middle income,MIC
6,Upper middle income,UMC


In [194]:
incomes = ['HIC', 'UMC', 'LMC', 'LIC','INX']
nivel_ingreso = pd.DataFrame()
for income in incomes:

    #Aqui vemos la lista de niveles de Ingreso y sus respectivos códigos
    url = 'http://api.worldbank.org/v2/country' 
    args = {
            "format":"json", 
            "prefix":"Getdata", 
            "incomelevel":income, 
            "date":"1990:2020"
            }
    
    datos = requests.get(url, params=args).json()[1]
    data = pd.json_normalize(datos)
    data = data[['id',
                'name',
                'region.value',
                'longitude',
                'latitude',
                'incomeLevel.value']]
    nivel_ingreso = pd.concat([nivel_ingreso, data], ignore_index=True)


In [195]:
nivel_ingreso

,id,name,region.value,longitude,latitude,incomeLevel.value
0,ABW,Aruba,Latin America & Caribbean,-70.0167,12.5167,High income
1,AND,Andorra,Europe & Central Asia,1.5218,42.5075,High income
2,ARE,United Arab Emirates,Middle East & North Africa,54.3705,24.4764,High income
3,ATG,Antigua and Barbuda,Latin America & Caribbean,-61.8456,17.1175,High income
4,AUS,Australia,East Asia & Pacific,149.129,-35.282,High income
...,...,...,...,...,...,...
174,TGO,Togo,Sub-Saharan Africa,1.2255,6.1228,Low income
175,UGA,Uganda,Sub-Saharan Africa,32.5729,0.314269,Low income
176,YEM,"Yemen, Rep.",Middle East & North Africa,44.2075,15.352,Low income
177,ZMB,Zambia,Sub-Saharan Africa,28.2937,-15.3982,Low income


## Obtención de distintos indicadores

In [3]:
def consultar(pais: str, indicador:str, pagina:int = 1):

    '''Función que contacta a la API y devuelve la respuesta
    que brinda la misma en formato json'''

    # Página de la api y path al recurso solicitado
    api_url = 'http://api.worldbank.org/v2/es'
    path = f'/country/{pais}/indicator/{indicador}'
    url = api_url + path

    # Creamos el diccionario con los parametros 
    # para el método get
    args= {
        "date":'1990:2020',
        'page':pagina,
        "per_page":1000,
        "format":"json",
        "prefix":"Getdata",
    }
   
    return requests.get(url,params=args).json()

In [4]:
def consultar_por_pais():
    '''Función que retorna un código de país,
    para, en caso que se quiera o se solicite,
    consultar en tiempo real la API'''
    paises=pd.read_parquet('../datasets/paises.parquet')

    seleccion = input("Ingrese un nombre de país: ")
    try:
        pais = paises.loc[paises.name.str.contains(seleccion, case=False),'id'].values[0]
    except:
        print("No se encontró el país, intente de nuevo")

In [217]:
# Estos indicadores fueron seleccionados 
# de forma manual al final del documento
indicadores = pd.read_csv('../datasets/indicadores_seleccionados.csv')

diccionario = { a:b for a,b in zip(indicadores.id.to_list(), 
          indicadores.name.to_list())}

In [95]:
def carga_incremental(pais, indicador):
    '''Función que a partir de un país y un indicador 
    llama a consultar y establece qué tipo de contenido tiene
    según eso devuelve o no un dataframe con todos los datos'''

    consulta = consultar(pais, indicador)
    try:
        # La primera parte de la respuesta nos indica en 
        # cuantas páginas de encuentra la información
        paginas = consulta[0]["pages"]

        # La segunda parte nos retorna una lista de 
        # diccionarios con la información que queríamos
        datos=consulta[1]

    except:
        print('No hay datos para:', indicador, pais)
        pass
    else:
        if paginas >= 1:
            # Agregamos los valores de las otras páginas a
            # nuestra lista de diccionarios
            for pagina in range(2,paginas+1):
                datos.extend(consultar(pais, indicador, pagina)[1])

            # Creo el DataFrame con todos los datos
            data = pd.json_normalize(datos)
            return data
        return pd.DataFrame(['error'],columns=['no_data'])


In [ ]:
for indicador in diccionario:
    print(indicador)
    datos = carga_incremental(pais='all', indicador=indicador)

    # Guardo el dataframe resultante
    datos.to_parquet(f'../datasets/df_TWB_{indicador}.parquet')
    print('Guardado')


# ETL

## Búsqueda de indicadores

La lista de indicadores bajo estudio se selecciona de forma manual a partir de los datos que indican distintas tesis de estudio. De todas formas los indicadores pasarán por un tamiz de análisis exploratorio

In [19]:
indicadores_todos = pd.read_csv('../datasets/indicadores.csv')

# Se completó los valores faltantes con 'na'
# para poder buscar sobre el dataframe
indicadores_todos.fillna('na',inplace=True)

In [37]:
interes = input("Indicador de interés: ")
indicadores_todos.loc[
     indicadores_todos.name.str.contains(interes,case=False),
     ['id','name','sourceNote']
     ]#.to_csv('temp.csv')

,id,name,sourceNote
17810,SP.POP.TOTL.FE.IN,"Población, mujeres","Población, mujeres"
17811,SP.POP.TOTL.FE.ZS,"Población, mujeres (% del total)",La población femenina es el porcentaje de la p...


In [26]:
interes = input("Indicador de interés: ")
seleccion = indicadores_todos.loc[indicadores_todos.id == interes,
     ['id','name','sourceNote']]
seleccion

,id,name,sourceNote
17815,SP.POP.TOTL.MA.ZS,"Población, hombres (% del total)",na


In [6]:
indicador_selec = pd.DataFrame()

In [108]:
indicador_selec = pd.concat([indicador_selec, seleccion], 
                            ignore_index=True)

## Exploración de los Datasets obtenidos

In [2]:
naciones_unidas = {
    "df_UNPD_mort_22" : "tasa_mortalidad_infantil",
    "df_UNPD_pop_54": "densidad_población_por_kilómetro_cuadrado)",
    "df_UNPD_imigrt_65": "migración_neta_total",
    "df_UNPD_pop_49": "población_total_por_sexo",
    "df_UNPD_mort_60": "total_muertes_por_sexo",
    "df_UNPD_pop_53": "tasa_bruta_cambio_natural_población",
    "df_UNPD_imigrt_66": "tasa_bruta_migración_neta",
    "df_UNPD_pop_72": "proporción_sexos_población_total",
    "df_UNPD_fam_1": "prevalencia_anticonceptivos_porcentaje",
    "df_UNPD_pop_67": "mediana_edad_población",
    "df_UNPD_mort_59": "tasa_bruta_mortalidad_por_1000_habitantes",
    "df_UNPD_pop_51": "tasa_bruta_variación_total_población",
    "df_UNPD_pop_50": "cambio_de_la_población",
    "df_UNPD_pop_41": "población_femenina_edad_reproductiva_(15-49 años)",
    "df_UNPD_mort_24": "tasa_mortalidad_menores_cinco_años",
    "df_UNPD_pop_52": "cambio_natural_población",
    "df_UNPD_fert_19": "tasa_fertilidad",
    "df_UNPD_marstat_42": "estado_civil_casado_porcentaje",
}

In [71]:
organizacion_salud = {'df_OMS_NUTRITION_ANAEMIA_CHILDREN_PREV': 'tasa_anemia_niños(%)',
'df_OMS_NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV': 'tasa_anemia_mujeres(%)',
'df_OMS_M_Est_cig_curr': 'tasa_consumo_cigarro(%)',
'df_OMS_SA_0000001688': 'tasa_consumo_alcohol(L)',
'df_OMS_NCD_BMI_30A' :'tasa_obesidad_pob(%)'}

In [3]:
banco_mundial = {
    'SP.DYN.LE00.IN': 'esperanza_vida_total',
    'SP.DYN.LE00.FE.IN': 'esperanza_vida_mujeres',
    'SP.DYN.LE00.MA.IN': 'esperanza_vida_varones',
    'SI.POV.GINI': 'índice_gini',
    'SE.XPD.TOTL.GD.ZS': 'gasto_púb_educacion_pje',
    'SE.COM.DURS': 'duración_educ_obligatoria',
    'NY.GDP.PCAP.CD': 'pib_pc_usd_actuales',
    'NY.GDP.MKTP.PP.CD': 'pib_ppa_prec_inter',
    'IQ.SCI.OVRL': 'capacidad_estadística',
    'SP.POP.TOTL.FE.ZS': 'población_mujeres_pje',
    'SP.POP.TOTL.MA.ZS': 'población_hombres_pje',
    'NY.GDP.PCAP.PP.CD': 'pib_pc_prec_inter',
    'AG.LND.FRST.ZS': 'porcentaje_de_bosque',
    'EN.ATM.CO2E.PC': 'emisiones_co2',
    'SH.XPD.CHEX.PC.CD': 'inversion_salud_percapita',
    'SH.MED.BEDS.ZS': 'camas_hospitales_c/1000personas',
    'SP.DYN.IMRT.IN': 'mortalidad_infantil_c/1000nacimientos',
    'SH.H2O.BASW.ZS': 'acceso_agua_potable(%)',
    'SH.STA.BASS.ZS': 'acceso_servicios_sanitarios(%)',
    'SH.STA.SUIC.P5': 'tasa_mortalidad_suicidio_c/100.000',
    'SL.UEM.TOTL.ZS': 'tasa_desempleo',
    'SP.URB.TOTL.IN.ZS': 'tasa_poblacion_urbana',
    'NY.GNP.PCAP.CD': 'INB_percapita'
}

In [11]:
problemas = ['df_UNPD_mort_22', 
            'df_UNPD_mort_24', 
            'df_UNPD_mort_60',
            'df_UNPD_pop_49']

In [18]:
import os
directorio = '../datasets/'

with os.scandir(directorio) as ficheros:

    # Tomamos unicamente la fecha y el iso3 para usarlo como indice
    df_twb=pd.read_parquet('../datasets/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]
    
    df_unpd = pd.read_parquet('../datasets/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]
    df_unpd.set_index(['countryiso3code', 'date'], inplace=True)

    df_oms = pd.read_parquet('../datasets/df_TWB_SP.DYN.LE00.IN.parquet')[['date','countryiso3code']]

    for fichero in ficheros:
        if fichero.name.startswith('df_TWB'):

            # obtengo el cógigo de indicador que se encuentra en el nombre del fichero
            codigo_fichero = fichero.name[7:-8]
            # busco el código en mi lista de códigos 
            # y procedo a renombrar la columna de interés
                
            df=pd.read_parquet(directorio+fichero.name)

            df_twb[banco_mundial[codigo_fichero]]=df.value
        
        elif fichero.name.startswith('df_UNPD'):
            codigo_fichero = fichero.name[:-8]
                
            if codigo_fichero in problemas:
                temp=pd.read_parquet(directorio+fichero.name)
                
                # Creo 3 tablas según el sexo sea hombre, mujer o ambos 
                # y selecciono las columnas de interés
                temp_male=temp.loc[temp.sex == "Male", ['iso3','timeLabel','value']]
                temp_female=temp.loc[temp.sex == "Female", ['iso3','timeLabel','value']]
                temp_both=temp.loc[temp.sex == "Both sexes", ['iso3','timeLabel','value']]

                # Renombro la columna de interés según el diccionario
                temp_both.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_ambos"}, inplace=True)
                temp_male.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_masc"}, inplace=True)
                temp_female.rename(columns={"value":f"{naciones_unidas[codigo_fichero]}_fem"}, inplace=True)

                # Asigno un index multiple
                temp_male.set_index(['iso3','timeLabel'], inplace=True)
                temp_female.set_index(['iso3','timeLabel'], inplace=True)
                temp_both.set_index(['iso3','timeLabel'], inplace=True)

                df_unpd = df_unpd.join(temp_both,
                    on=['countryiso3code','date']
                    )

                df_unpd = df_unpd.join(temp_male,
                    on=['countryiso3code','date']
                    )

                df_unpd = df_unpd.join(temp_female,
                    on=['countryiso3code','date']
                    )

            else:
                temp=pd.read_parquet(directorio+fichero.name)

                temp.set_index(['iso3','timeLabel'], inplace=True)

                temp.rename(columns={"value":naciones_unidas[codigo_fichero]}, inplace=True)
                
                df_unpd = df_unpd.join(temp[[naciones_unidas[codigo_fichero]]], 
                                        on=['countryiso3code','date'])

Al usar como tabla base una tabla con más valores de los que poseían las tablas del banco mundial, nos queda una tabla con valores faltantes en todas sus filas. Esas filas con todos los datos faltantes son las que se procede a eliminar a continuación

In [19]:
df_unpd.dropna(how='all',inplace=True)

In [21]:
df_twb

,date,countryiso3code,acceso_agua_potable(%),porcentaje_de_bosque,tasa_mortalidad_suicidio_c/100.000,mortalidad_infantil_c/1000nacimientos,pib_ppa_prec_inter,población_mujeres_pje,emisiones_co2,tasa_poblacion_urbana,...,INB_percapita,duración_educ_obligatoria,índice_gini,esperanza_vida_varones,gasto_púb_educacion_pje,acceso_servicios_sanitarios(%),capacidad_estadística,camas_hospitales_c/1000personas,pib_pc_prec_inter,pib_pc_usd_actuales
0,2020,AFE,59.511910,30.174186,NaN,42.447198,2.478218e+12,50.364579,NaN,36.783306,...,1397.543390,7.0,NaN,62.121491,4.608170,30.931064,57.632861,NaN,3659.272868,1360.878645
1,2019,AFE,58.782401,30.391558,8.036293,43.591252,2.517975e+12,50.373008,0.913618,36.291322,...,1506.731921,7.0,NaN,61.813185,4.846845,30.742610,58.502425,NaN,3814.846625,1511.309259
2,2018,AFE,57.860301,30.611444,8.128884,44.795221,2.418685e+12,50.382358,0.917507,35.807770,...,1464.995375,7.0,NaN,61.470354,4.951635,30.384759,58.067643,NaN,3761.035160,1541.031661
3,2017,AFE,56.921012,30.824248,8.244062,46.108140,2.299420e+12,50.392615,0.937926,35.332373,...,1459.517554,7.0,NaN,61.083445,4.963090,30.114592,59.710148,NaN,3670.891196,1629.404273
4,2016,AFE,55.963351,31.039613,8.324659,47.527244,2.212683e+12,50.403727,0.948410,34.865101,...,1475.979716,7.0,NaN,60.643170,4.821875,29.618405,58.599045,NaN,3627.474284,1446.533624
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,1994,ZWE,NaN,48.190255,NaN,55.000000,2.165871e+10,50.486345,1.417186,31.335000,...,630.000000,NaN,NaN,49.136000,44.333981,NaN,NaN,NaN,1923.209711,611.865276
8242,1993,ZWE,NaN,48.309345,NaN,53.700000,1.941304e+10,50.412002,1.539741,30.940000,...,640.000000,NaN,NaN,50.663000,NaN,NaN,NaN,NaN,1750.061859,591.719682
8243,1992,ZWE,NaN,48.428435,NaN,52.400000,1.876622e+10,50.343007,1.694416,30.499000,...,700.000000,NaN,NaN,52.210000,22.322210,NaN,NaN,NaN,1721.591128,619.372083
8244,1991,ZWE,NaN,48.547525,NaN,51.200000,2.016618e+10,50.285057,1.713321,29.738000,...,850.000000,NaN,NaN,53.717000,NaN,NaN,NaN,NaN,1888.041180,809.051140


## Sección del desarrollador

In [19]:
select = 'df_UNPD_pop_49.parquet'
temp = pd.read_parquet(f'../datasets/{select}')

#temp_male=temp.loc[temp.sex == "Male", ['iso3','timeLabel','value']]

temp[['iso3','timeLabel','value']]
#temp.set_index(['iso3','timeLabel'], inplace=True)

#temp.Dim1.value_counts()

,iso3,timeLabel,value
0,AFG,1990,5348387
1,AFG,1990,5346409
2,AFG,1990,10694796
3,AFG,1991,5372960
4,AFG,1991,5372208
...,...,...,...
21013,ZMB,2019,9314079
21014,ZMB,2019,18380477
21015,ZMB,2020,9338614
21016,ZMB,2020,9589102


In [ ]:
# Creo 3 tablas según el sexo sea hombre, mujer o ambos 
# y selecciono las columnas de interés
temp_severa=temp.loc[temp.sex == "SEVERE", ['SpatialDim','timeLabel','NumericValue']]
temp_moderada=temp.loc[temp.sex == "MODERATE", ['SpatialDim','timeLabel','NumericValue']]
temp_leve=temp.loc[temp.sex == "MILD", ['SpatialDim','timeLabel','NumericValue']]

# Renombro la columna de interés según el diccionario
temp_leve.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)
temp_severa.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)
temp_moderada.rename(
    columns={"NumericValue":organizacion_salud[codigo_fichero]}, 
    inplace=True)

# Asigno un index multiple
temp_severa.set_index(['SpatialDim','timeLabel'], inplace=True)
temp_moderada.set_index(['SpatialDim','timeLabel'], inplace=True)
temp_leve.set_index(['SpatialDim','timeLabel'], inplace=True)

df_oms = df_oms.join(temp_leve,
    on=['countryiso3code','date']
    )

df_oms = df_oms.join(temp_severa,
    on=['countryiso3code','date']
    )

df_oms = df_oms.join(temp_moderada,
    on=['countryiso3code','date']
    )

# Clasificación según nivel de ingreso per cápita

In [242]:
temp= pd.DataFrame()
temp= pd.cut(trabajo['INB_percapita'],
                bins=[0,1086,4256,13206,121901],
                labels=['low_income', 
                        'lower_middle_income', 
                        'upper_middle_income',
                        'high_income'],
                include_lowest = True)

# Analisis exploratorio

Al ser nuestro objetivo la esperanza de vida, no sería recomendable inventar esos datos con un promedio, ni nada parecido. por lo que vamos a limpiar todas las filas que tengan valores nulos en la esperanza de vida al nacer

In [46]:
trabajo.dropna(subset=['esperanza_vida_total'], inplace=True)

Verificamos nuevamente los datos

In [60]:
trabajo.isna().sum()

date                            0
countryiso3code                 0
pib_ppa_prec_inter            501
población_mujeres_pje         166
esperanza_vida_total            0
población_hombres_pje         166
duración_educ_obligatoria    1858
índice_gini                  3772
esperanza_vida_mujeres          0
esperanza_vida_varones          0
gasto_púb_educacion_pje      2414
capacidad_estadística        3118
pib_pc_prec_inter             504
pib_pc_usd_actuales           276
id                              0
incomeLevel.value               0
dtype: int64

Vamos a eliminar todas aquellas columnas que tengan más del 30% de datos núlos. Dado que de trabajar sobre ellas con alguna estratégia de imputación no sería adecuado.

In [78]:
(trabajo.isna().sum()/trabajo.shape[0])*100

date                          0.000000
countryiso3code               0.000000
pib_ppa_prec_inter            9.553776
población_mujeres_pje         3.165523
esperanza_vida_total          0.000000
población_hombres_pje         3.165523
duración_educ_obligatoria    35.430969
índice_gini                  71.929825
esperanza_vida_mujeres        0.000000
esperanza_vida_varones        0.000000
gasto_púb_educacion_pje      46.033562
capacidad_estadística        59.458429
pib_pc_prec_inter             9.610984
pib_pc_usd_actuales           5.263158
id                            0.000000
incomeLevel.value             0.000000
dtype: float64

In [79]:
trabajo.drop(columns=[  'índice_gini',
                        'capacidad_estadística', 
                        'gasto_púb_educacion_pje', 
                        'duración_educ_obligatoria'],
            inplace=True
            )

In [82]:
trabajo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5244 entries, 0 to 5548
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    5244 non-null   object 
 1   countryiso3code         5244 non-null   object 
 2   pib_ppa_prec_inter      4743 non-null   float64
 3   población_mujeres_pje   5078 non-null   float64
 4   esperanza_vida_total    5244 non-null   float64
 5   población_hombres_pje   5078 non-null   float64
 6   esperanza_vida_mujeres  5244 non-null   float64
 7   esperanza_vida_varones  5244 non-null   float64
 8   pib_pc_prec_inter       4740 non-null   float64
 9   pib_pc_usd_actuales     4968 non-null   float64
 10  id                      5244 non-null   object 
 11  incomeLevel.value       5244 non-null   object 
dtypes: float64(8), object(4)
memory usage: 532.6+ KB


In [88]:
corr_matrix = trabajo.corr()
corr_matrix["esperanza_vida_total"].sort_values(ascending=False)

esperanza_vida_total      1.000000
esperanza_vida_mujeres    0.994401
esperanza_vida_varones    0.993782
pib_pc_prec_inter         0.591415
pib_pc_usd_actuales       0.540825
pib_ppa_prec_inter        0.211011
población_hombres_pje     0.032236
población_mujeres_pje    -0.032236
Name: esperanza_vida_total, dtype: float64

Dividimos el dataset de trabajo en cuatro datasets para analizar según su nivel de ingresos

In [90]:
trabajo['incomeLevel.value'].value_counts()

Lower middle income    1550
Upper middle income    1427
High income            1368
Low income              868
Not classified           31
Name: incomeLevel.value, dtype: int64

In [91]:
ingreso_bajo = trabajo.loc[trabajo['incomeLevel.value'] == 'Low income']
ingreso_medio_bajo = trabajo.loc[trabajo['incomeLevel.value'] == 'Lower middle income']
ingreso_medio_alto = trabajo.loc[trabajo['incomeLevel.value'] == 'Upper middle income']
ingreso_alto = trabajo.loc[trabajo['incomeLevel.value'] == 'High income']


In [93]:
ingreso_alto.isnull().sum()

date                        0
countryiso3code             0
pib_ppa_prec_inter        224
población_mujeres_pje     148
esperanza_vida_total        0
población_hombres_pje     148
esperanza_vida_mujeres      0
esperanza_vida_varones      0
pib_pc_prec_inter         227
pib_pc_usd_actuales       105
id                          0
incomeLevel.value           0
dtype: int64